In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define Paths
IMAGE_FOLDER = "/content/drive/MyDrive/data/Ball 1/Drop 1"
OUTPUT_FOLDER = "outputs"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Get Sorted List of Image Files
images = sorted([f for f in os.listdir(IMAGE_FOLDER) if f.endswith(".bmp")])
if not images:
    print("❌ Error: No images found in the directory.")
    exit()

print(f"✅ Starting detection from first frame: {images[0]}")

# Initialize Tracking Variables
last_known_position = None
last_known_size = None
deformation_data = []

# Football Deformation Detection Function with Segmentation
def detect_football_deformation(gray_image, original_image, last_known_position, last_known_size):
    detected_center = None
    detected_size = None

    # Apply Segmentation
    # Otsu's thresholding to automatically find the threshold
    _, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Morphological operations to remove noise and close gaps
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Apply Gaussian Blur to the thresholded image
    blurred = cv2.GaussianBlur(thresh, (5, 5), 0)

    # Detect Edges and Contours
    edges = cv2.Canny(blurred, 30, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(f"Detected {len(contours)} contours after segmentation.")

    # Save thresholded image for debugging
    cv2.imwrite(os.path.join(OUTPUT_FOLDER, "segmented_mask.jpg"), thresh)

    if contours:
        # Sort contours by area (largest first)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)

        for contour in contours[:3]:  # Check top 3 largest contours
            area = cv2.contourArea(contour)
            if area < 500 or len(contour) < 5:  # Minimum area and points for ellipse
                continue

            # Calculate circularity to filter football-like shapes
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
            if circularity < 0.5:  # Too far from a circle
                continue

            # Fit ellipse
            ellipse = cv2.fitEllipse(contour)
            detected_center = (int(ellipse[0][0]), int(ellipse[0][1]))
            detected_size = (int(ellipse[1][0]), int(ellipse[1][1]))  # Width, Height

            # Check continuity with last known position (if available)
            if last_known_position:
                distance = np.sqrt((detected_center[0] - last_known_position[0])**2 +
                                 (detected_center[1] - last_known_position[1])**2)
                if distance > 100:  # Too far from last position
                    continue

            # Draw the detected football
            cv2.ellipse(original_image, ellipse, (0, 255, 0), 2)  # Green ellipse
            cv2.circle(original_image, detected_center, 3, (0, 0, 255), -1)  # Red center
            cv2.line(original_image,
                    (detected_center[0] - detected_size[0]//2, detected_center[1]),
                    (detected_center[0] + detected_size[0]//2, detected_center[1]),
                    (0, 255, 255), 2)  # Yellow diameter

            # Calculate deformation ratio
            deformation_ratio = min(detected_size) / max(detected_size)
            print(f"⚽ Football detected at {detected_center}, size {detected_size}, deformation {deformation_ratio:.2f}")

            # Store data
            deformation_data.append([detected_center[0], detected_center[1], detected_size[0], detected_size[1], deformation_ratio])
            return detected_center, detected_size, original_image

    # Fallback to last known position if detection fails
    if last_known_position and last_known_size:
        print("⚠️ Using last known position.")
        cv2.circle(original_image, last_known_position, last_known_size[0]//2, (255, 0, 0), 2)  # Blue circle
        return last_known_position, last_known_size, original_image

    print("⚠️ No football detected.")
    return None, None, original_image

# Process All Images
for idx, image_name in enumerate(images):
    image_path = os.path.join(IMAGE_FOLDER, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"❌ Skipping unreadable image: {image_path}")
        continue

    print(f"🔍 Processing: {image_name}")
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect Football Deformation
    detected_center, detected_size, detected_image = detect_football_deformation(
        gray_image, image, last_known_position, last_known_size
    )

    # Update last known values
    if detected_center and detected_size:
        last_known_position = detected_center
        last_known_size = detected_size

        # Save and Display
        detected_output = os.path.join(OUTPUT_FOLDER, f"football_deformed_{image_name}")
        cv2.imwrite(detected_output, detected_image)

        plt.figure(figsize=(6, 10))
        plt.imshow(cv2.cvtColor(detected_image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"Deformed Football - {image_name}")
        plt.show(block=False)
        plt.pause(0.5)
        plt.clf()

# Save Deformation Data to CSV
df = pd.DataFrame(deformation_data, columns=["X", "Y", "Width", "Height", "Deformation_Ratio"])
df.to_csv(os.path.join(OUTPUT_FOLDER, "football_deformation_data.csv"), index=False)

print("🎉 Football Deformation Detection Completed!")

# --- Video Creation Section ---
print("📹 Starting video creation...")

# Get Sorted List of Processed Image Files
processed_images = sorted([f for f in os.listdir(OUTPUT_FOLDER) if f.startswith("football_deformed_") and f.endswith(".bmp")])
if not processed_images:
    print("❌ Error: No processed images found in the output folder.")
    print("Please ensure the deformation detection ran successfully.")
    exit()

print(f"✅ Found {len(processed_images)} processed images for video.")

# Read the first image to get dimensions
first_image_path = os.path.join(OUTPUT_FOLDER, processed_images[0])
frame = cv2.imread(first_image_path)
if frame is None:
    print("❌ Error: Could not read the first image.")
    exit()
height, width, layers = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 codec
fps = 30  # Frames per second (adjust based on your image sequence timing)
video_path = os.path.join(OUTPUT_FOLDER, "football_deformation_video.mp4")
out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# Write each processed image to the video
for image_name in processed_images:
    image_path = os.path.join(OUTPUT_FOLDER, image_name)
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"❌ Skipping unreadable frame: {image_path}")
        continue
    out.write(frame)
    print(f"📝 Added frame: {image_name}")

# Release the video writer
out.release()
print(f"🎉 Video saved as {video_path}")

# Optional: Display a message to open the video
print("✅ Video creation complete! Open 'football_deformation_video.mp4' with any media player to view the results.")
